# Wikipedia Edits Data Pipeline

This notebook demonstrates a simple ETL pipeline that:
1. Extracts data from the Wikipedia REST API
2. Transforms it to JSON Lines format
3. Uploads directly to S3

In [25]:
# Set username
USERNAME = "petra"

## Setup and Imports

In [26]:
import datetime
import json

import boto3
import requests

## Extract: Retrieve Data from Wikipedia API

We use the Wikimedia Analytics API to fetch the most edited pages for a specific date. The API returns JSON with page titles and edit counts.

**API Documentation:** https://doc.wikimedia.org/generated-data-platform/aqs/analytics-api/reference/page-views.html

In [27]:
# Try different dates to see how the data changes
DATE_PARAM = "2025-11-27"

date = datetime.datetime.strptime(DATE_PARAM, "%Y-%m-%d")

# Construct the API URL
url = f"https://wikimedia.org/api/rest_v1/metrics/pageviews/top/en.wikipedia.org/all-access/{date.strftime('%Y/%m/%d')}"
print(f"Requesting REST API URL: {url}")

# Make the API request
wiki_server_response = requests.get(url, headers={"User-Agent": "curl/7.68.0"})
wiki_response_status = wiki_server_response.status_code
wiki_response_body = wiki_server_response.text

print(f"Wikipedia REST API Response body: {wiki_response_body[:500]}...")
print(f"Wikipedia REST API Response Code: {wiki_response_status}")

# Validate response
if wiki_response_status != 200:
    raise Exception(f"Received non-OK status code from Wiki Server: {wiki_response_status}")
print(f"Successfully retrieved Wikipedia data, content-length: {len(wiki_response_body)}")

Requesting REST API URL: https://wikimedia.org/api/rest_v1/metrics/pageviews/top/en.wikipedia.org/all-access/2025/11/27
Wikipedia REST API Response body: {"items":[{"project":"en.wikipedia","access":"all-access","year":"2025","month":"11","day":"27","articles":[{"article":"Main_Page","views":5988101,"rank":1},{"article":"Special:Search","views":797249,"rank":2},{"article":"Stranger_Things_season_5","views":655701,"rank":3},{"article":"Stranger_Things","views":353714,"rank":4},{"article":"Jack_White","views":349384,"rank":5},{"article":"Google_Chrome","views":306639,"rank":6},{"article":"Wikipedia:Featured_pictures","views":248728,"rank":7},{"arti...
Wikipedia REST API Response Code: 200
Successfully retrieved Wikipedia data, content-length: 55549


## Transform: Process Raw Data into JSON Lines

Convert the raw API response into a structured JSON Lines format suitable for analytics. Each line is a valid JSON object representing one page's edit statistics.

In [28]:
# Parse the API response and extract top edits
wiki_response_parsed = wiki_server_response.json()
top_views = wiki_response_parsed["items"][0]["articles"]

# Transform to JSON Lines format
current_time = datetime.datetime.now(datetime.timezone.utc)
json_lines = ""
for page in top_views[:6]:
    record = {
        "title": page["article"],
        "views": page["views"],
        "rank": page["rank"],
        "date": date.strftime("%Y-%m-%d"),
        "retrieved_at": current_time.replace(tzinfo=None).isoformat(),
    }
    json_lines += json.dumps(record) + "\n"

print(f"Transformed {len(top_views)} records to JSON Lines")
print(f"First few lines:\n{json_lines[:500]}...")

Transformed 1000 records to JSON Lines
First few lines:
{"title": "Main_Page", "views": 5988101, "rank": 1, "date": "2025-11-27", "retrieved_at": "2025-12-10T16:55:24.298276"}
{"title": "Special:Search", "views": 797249, "rank": 2, "date": "2025-11-27", "retrieved_at": "2025-12-10T16:55:24.298276"}
{"title": "Stranger_Things_season_5", "views": 655701, "rank": 3, "date": "2025-11-27", "retrieved_at": "2025-12-10T16:55:24.298276"}
{"title": "Stranger_Things", "views": 353714, "rank": 4, "date": "2025-11-27", "retrieved_at": "2025-12-10T16:55:24.298276...


---
## S3 Bucket

In [29]:
S3_WIKI_BUCKET = "petra-wikidata"
s3 = boto3.client("s3")

# Check Bucket exisits

try:
    s3.head_bucket(Bucket=S3_WIKI_BUCKET)
    print(f"Bucket {S3_WIKI_BUCKET} exists!")
except Exception as e:
    print(f"Bucket {S3_WIKI_BUCKET} not found: {e}")
    raise

Bucket petra-wikidata exists!


---
## Upload JSON Lines to S3

**Requirements:**
- Use `s3.put_object()` to upload the data directly
- Place the file under `raw-views/` prefix in S3
- File name: `raw-edits-views-MM-DD.json` (use the date from `DATE_PARAM`)

**Documentation:** [put_object](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3/client/put_object.html)

In [30]:
s3_key = f"raw-views/raw-views-{date.strftime('%Y-%m-%d')}.json"
try:
    s3.put_object(Body=json_lines, Bucket=S3_WIKI_BUCKET, Key=s3_key)
    print(f"Uploaded data to s3://{s3_key}")
except Exception as e:
    print(f"Error uploading data to s3: {e}")
    raise


Uploaded data to s3://raw-views/raw-views-2025-11-27.json


In [31]:
expected_key = f"raw-views/raw-views-{date.strftime('%Y-%m-%d')}.json"
try:
    s3.head_object(Bucket=S3_WIKI_BUCKET, Key=expected_key)
    print(f"File uploaded successfully to s3://{S3_WIKI_BUCKET}/{expected_key}")
except Exception as e:
    print(f"File not found at s3://{S3_WIKI_BUCKET}/{expected_key}")
    raise

File uploaded successfully to s3://petra-wikidata/raw-views/raw-views-2025-11-27.json
